In [1]:
import subprocess
import fnmatch
from pathlib import Path

import geowombat as gw
from geowombat.util import GeoDownloads
from geowombat.radiometry import BRDF, RadTransforms, landsat_pixel_angles
from geowombat.data import rgbn

import numpy as np
import pandas as pd
import xarray as xr
import dask.array as da

## Download data

In [14]:
gdl = GeoDownloads()

## Landsat (ETM+)

In [15]:
gdl.list_gcp('l7', '225/083/*225083_2019*_T*')

del_keys = [k for k, v in gdl.search_dict.items() if 'gap_mask' in k]

for dk in del_keys:
    del gdl.search_dict[dk]

In [16]:
list(gdl.search_dict.keys())

['LE07/01/225/083/LE07_L1GT_225083_20190107_20190204_01_T2',
 'LE07/01/225/083/LE07_L1GT_225083_20190123_20190220_01_T2',
 'LE07/01/225/083/LE07_L1GT_225083_20190224_20190324_01_T2',
 'LE07/01/225/083/LE07_L1GT_225083_20190312_20190409_01_T2',
 'LE07/01/225/083/LE07_L1GT_225083_20190616_20190713_01_T2',
 'LE07/01/225/083/LE07_L1TP_225083_20190208_20190306_01_T1',
 'LE07/01/225/083/LE07_L1TP_225083_20190328_20190423_01_T1',
 'LE07/01/225/083/LE07_L1TP_225083_20190413_20190509_01_T1',
 'LE07/01/225/083/LE07_L1TP_225083_20190429_20190525_01_T1',
 'LE07/01/225/083/LE07_L1TP_225083_20190515_20190610_01_T1',
 'LE07/01/225/083/LE07_L1TP_225083_20190531_20190627_01_T1',
 'LE07/01/225/083/LE07_L1TP_225083_20190702_20190728_01_T1',
 'LE07/01/225/083/LE07_L1TP_225083_20190718_20190813_01_T1',
 'LE07/01/225/083/LE07_L1TP_225083_20190803_20190829_01_T1',
 'LE07/01/225/083/LE07_L1TP_225083_20190803_20190831_01_T1',
 'LE07/01/225/083/LE07_L1TP_225083_20190819_20190914_01_T1',
 'LE07/01/225/083/LE07_L

In [ ]:
gdl.search_dict['LE07/01/225/083/LE07_L1TP_225083_20190208_20190306_01_T1']

In [17]:
search_wildcards = ['ANG.txt', 'MTL.txt', 'B8.TIF']
outdir = '/Users/Admin/Documents/jordan/imagery'
p = Path(outdir)

In [18]:
file_info = gdl.download_gcp('l7',
                             outdir=outdir, 
                             downloads='LE07/01/225/083/LE07_L1TP_225083_20190208_20190306_01_T1',
                             search_wildcards=search_wildcards,
                             verbose=1)

In [ ]:
with gw.config.update():

    with gw.open(p.joinpath('LE07_L1TP_225083_20190208_20190306_01_T1_B8.TIF').as_posix(),
                 band_names=['pan']) as ds:
        print(ds)
        #ds.gw.imshow(band_names='pan')
        ds.sel(band='pan')[5000:6000, 5000:6000].plot.imshow(robust=True)

## Landsat (OLI TIRS)

#### 2018

In [ ]:
gdl.list_gcp('l8', '225/083/*225083_201809*_T*')

In [ ]:
list(gdl.search_dict.keys())

#### 2019

In [ ]:
gdl.list_gcp('l8', '225/083/*225083_2019*_T*')

In [ ]:
list(gdl.search_dict.keys())

In [ ]:
# gdl.search_dict['LC08/01/225/083/LC08_L1TP_225083_20190131_20190206_01_T1']

In [ ]:
gdl.search_dict['LC08/01/225/083/LC08_L1TP_225083_20180909_20180912_01_T1']

In [ ]:
file_info = gdl.download_gcp('l8',
                             outdir=outdir, 
                             downloads='LC08/01/225/083/LC08_L1TP_225083_20180909_20180912_01_T1',
                             search_wildcards=search_wildcards,
                             verbose=1)

In [ ]:
search_wildcards = ['ANG.txt', 'MTL.txt', 'B2.TIF', 'B3.TIF', 'B4.TIF', 'B5.TIF']
outdir = '/media/jcgr/data/imagery/google/p225r083/downloads'

In [ ]:
file_info = gdl.download_gcp('l8',
                             outdir=outdir, 
                             search_wildcards=search_wildcards,
                             verbose=1)

In [ ]:
# file_info

## DN -> SR -> BRDF

In [5]:
rt = RadTransforms()
br = BRDF()

In [10]:
outdir_angles = p.joinpath('angles')
outdir_brdf = p.joinpath('brdf')
gw_bin = Path('/Users/Admin/Documents/jordan/scripts/Python/geowombat/geowombat/bin')

In [11]:
angle_infos = dict()

kwargs = dict(readxsize=1024, 
              readysize=1024,
              verbose=1,
              separate=False,
              n_workers=1, 
              n_threads=8,
              n_chunks=100,
              overwrite=False)

In [20]:
file_info

{'LE07/01/225/083/LE07_L1TP_225083_20190208_20190306_01_T1': {'angle': FileInfo(name='/Users/Admin/Documents/jordan/imagery/LE07_L1TP_225083_20190208_20190306_01_T1_ANG.txt', key='angle'),
  'meta': FileInfo(name='/Users/Admin/Documents/jordan/imagery/LE07_L1TP_225083_20190208_20190306_01_T1_MTL.txt', key='meta'),
  'B8': FileInfo(name='/Users/Admin/Documents/jordan/imagery/LE07_L1TP_225083_20190208_20190306_01_T1_B8.TIF', key='B8')}}

In [24]:
for finfo_key, finfo_dict in file_info.items():
    
#     if 'LC08_L1TP_225083_20190216_20190222_01_T1' not in finfo_key:
#         continue

    meta = rt.get_landsat_coefficients(finfo_dict['meta'].name)
    
    angle_info = landsat_pixel_angles(finfo_dict['angle'].name, 
                                      finfo_dict['B8'].name, 
                                      outdir_angles.as_posix(), 
                                      meta.sensor, 
                                      l57_angles_path=gw_bin.joinpath('/ESPA/landsat_angles').as_posix(), 
                                      l8_angles_path=gw_bin.joinpath('/ESPA/l8_angles').as_posix(),
                                      verbose=1)
    
    out_dn = outdir_brdf.joinpath(Path(finfo_key).name + '_dn.tif').as_posix()
    out_sr = outdir_brdf.joinpath(Path(finfo_key).name + '_sr.tif').as_posix()
    out_brdf = outdir_brdf.joinpath(Path(finfo_key).name + '.tif').as_posix()
    break
    #with gw.config.update(ref_res=(100, 100)):

    with gw.open(angle_info.sza) as sza, gw.open(angle_info.vza) as vza, gw.open(angle_info.saa) as saa, gw.open(angle_info.vaa) as vaa:

        with gw.open([finfo_dict['B2'].name, 
                      finfo_dict['B3'].name, 
                      finfo_dict['B4'].name, 
                      finfo_dict['B5'].name], 
                     band_names=['blue', 'green', 'red', 'nir'],
                     stack_dim='band') as dn:

            #dn.gw.to_raster(out_dn, **kwargs)    

            # Convert DN to surface reflectance
            dn_sr = rt.dn_to_sr(dn, 
                                sza, saa, vza, vaa, 
                                sensor=meta.sensor, 
                                meta=meta)

            #dn_sr.gw.to_raster(out_sr, **kwargs)

            # BRDF normalization
            dn_sr_brdf = br.norm_brdf(dn_sr, 
                                      sza, saa, vza, vaa, 
                                      sensor=meta.sensor, 
                                      wavelengths=dn.band.values.tolist(), 
                                      nodata=0)

            dn_sr_brdf.gw.to_raster(out_brdf, **kwargs)    

    angle_infos[finfo_key] = angle_info

19:11:29:INFO:487:angles.landsat_pixel_angles:  Generating pixel angles ...


IndexError: list index out of range

In [ ]:
for finfo_key, finfo_dict in file_info.items():
    
    out_brdf = Path(outdir_brdf).joinpath(Path(finfo_key).name + '.tif').as_posix()
    
    out_brdf_ovr = out_brdf + '.ovr'
    
    if not Path(out_brdf_ovr).is_file():

        print(out_brdf)

        com = 'gdaladdo -ro {} 2 4 8 16'.format(out_brdf)
        subprocess.call(com, shell=True)

## Get coefficients from metadata

In [ ]:
rt = RadTransforms()

meta = rt.get_landsat_coefficients('/Users/Admin/Documents/jordan/imagery/LE07_L1GT_225083_20190107_20190204_01_T2_MTL.txt')

In [ ]:
meta

In [ ]:
rt = RadTransforms()

# meta = rt.get_coefficients(file_info[1].name)
meta = rt.get_landsat_coefficients('/media/jcgr/data/imagery/temp/google/LC08_L1TP_042034_20160121_20170224_01_T1_MTL.txt')

In [ ]:
meta

## Check data

In [ ]:
# with gw.open([file_info[2].name, file_info[3].name], 
#              band_names=['red', 'nir'],
#              stack_dim='band') as ds:
#     print(ds)
# #     ds.sel(band='nir').plot.imshow()

## Generate pixel angle files

In [ ]:
# angle_info = gw.gen_pixel_angles(file_info[0].name, 
#                                  file_info[2].name, 
#                                  '/media/jcgr/data/imagery/temp/google/angles', 
#                                  meta.sensor, 
#                                  l57_angles_path='/home/jcgr/Documents/scripts/Python/git_repos/geowombat/bin/ESPA/landsat_angles', 
#                                  l8_angles_path='/home/jcgr/Documents/scripts/Python/git_repos/geowombat/bin/ESPA/l8_angles',
#                                  verbose=1)

In [ ]:
# angle_info

In [ ]:
# with gw.open(angle_info.senze) as ds:
#     ds.sel(band=1).plot.imshow(robust=True)

In [ ]:
with gw.open(angle_info.senaz) as ds:
    ds.sel(band=1).plot.imshow(robust=True)

In [ ]:
with gw.open(angle_info.solaz) as solaz:
    with gw.open(angle_info.senaz) as senaz:
        rel_azimuth_angle = xr.ufuncs.fabs(solaz - senaz - 180.0)
        rel_azimuth_angle.sel(band=1).plot.imshow(robust=True)

## Convert to surface reflectance

In [ ]:
angle_info

In [ ]:
# with gw.open(angle_info.sze) as sze:
#     with gw.open(angle_info.vze) as vze:
#         with gw.open(angle_info.saa) as saa:
#             with gw.open(angle_info.vaa) as vaa:
#                 with gw.open([file_info[2].name, file_info[3].name], 
#                              band_names=['red', 'nir'],
#                              stack_dim='band') as dn:

kwargs = dict(readxsize=1024, 
              readysize=1024, 
              n_workers=1, 
              n_threads=4,
              n_chunks=100,
              overwrite=True)

with gw.open('/media/jcgr/data/imagery/temp/google/angles/LC08_L1TP_042034_20160121_20170224_01_T1_solar_zenith.tif', chunks=512) as sze:
    with gw.open('/media/jcgr/data/imagery/temp/google/angles/LC08_L1TP_042034_20160121_20170224_01_T1_sensor_zenith.tif', chunks=512) as vze:
        with gw.open('/media/jcgr/data/imagery/temp/google/angles/LC08_L1TP_042034_20160121_20170224_01_T1_solar_azimuth.tif', chunks=512) as saa:
            with gw.open('/media/jcgr/data/imagery/temp/google/angles/LC08_L1TP_042034_20160121_20170224_01_T1_sensor_azimuth.tif', chunks=512) as vaa:                
                
                with gw.open(['/media/jcgr/data/imagery/temp/google/LC08_L1TP_042034_20160121_20170224_01_T1_B4.TIF', 
                              '/media/jcgr/data/imagery/temp/google/LC08_L1TP_042034_20160121_20170224_01_T1_B5.TIF',
                              '/media/jcgr/data/imagery/temp/google/LC08_L1TP_042034_20160121_20170224_01_T1_B6.TIF'], 
                             band_names=['red', 'nir', 'swir1'],
                             stack_dim='band',
                             chunks=512) as dn:
                    
#                     dn.gw.to_raster('/media/jcgr/data/imagery/temp/google/dn.tif', **kwargs)

                    # Convert DN to surface reflectance
                    dn_sr = rt.dn_to_sr(dn, sze, saa, vze, vaa, sensor=meta.sensor, meta=meta)
                    
#                     dn_sr.gw.to_raster('/media/jcgr/data/imagery/temp/google/dn_sr.tif', **kwargs)
                    
                    # BRDF normalization
                    dn_sr_brdf = BRDF().norm_brdf(dn_sr, sze, saa, vze, vaa, sensor=meta.sensor, wavelengths=dn.band.values.tolist(), nodata=0)
                    
#                     dn_sr_brdf.gw.to_raster('/media/jcgr/data/imagery/temp/google/dn_sr_brdf.tif', **kwargs)

In [ ]:
dn_sr

In [ ]:
dn_sr_brdf

In [ ]:
dn.sel(band='nir').plot.imshow(robust=True)

In [ ]:
dn_sr.sel(band='nir').plot.imshow(robust=True)

In [ ]:
dn_sr_brdf.sel(band='nir').plot.imshow(robust=True)